In [ ]:
import pandas as pd
import numpy as np
import scipy.optimize as opt

In [ ]:
class Leaf:
    def __init__(self,value):
        self.value = value

In [ ]:
class Node:
    def __init__(self,branches,attribute,threshold):
        self.branches = branches
        self.threshold = threshold
        self.attribute = attribute
        
    def get(self,df):
        if isinstance(df[self.attribute],(int,float)):
            return self.branches[0] if df[self.attribute] < self.threshold else self.branches[1]
        else:
            return self.branches[0] if df[self.attribute] in self.threshold else self.branches[1]
        

In [ ]:
class Tree:
    def __init__(self,root):
        self.root = root
        
    def predict(self,x):
        item = self.root
        while isinstance(item,Node):
            item = item.get(x)
        return item

In [ ]:
r=Node([Leaf('young'),Leaf('old')],"age",18)
t=Tree(r)
print(t.predict({"age":2}).value)

In [ ]:
print(t.predict({"age":20}).value)

In [ ]:
class CART:
    def __init__(self,df,y_name,X_names):
        self.df = df
        self.y_name = y_name
        self.X_names = X_names
        self.tree = None
        self.splittyness = 1.
        self.leaf_loss_threshold = 1e-12
        
        self.classes = np.unique(df[self.y_name]).tolist()
        n = len(self.classes)
        self.confusion_matrix = np.zeros((n,n))
        
    def create_tree(self,splittyness=1., leaf_loss_threshold=1e-12):
        self.splittyness = splittyness
        self.leaf_loss_threshold = leaf_loss_threshold
        root = self._node_or_leaf(self.df)
        self.tree = Tree(root)
        return self.tree
    
    def _gini_impurity(self, df):
        unique, counts = np.unique(df[self.y_name].values, return_counts=True)
        N = df[self.y_name].values.ravel().size
        p = counts/N
        #print(unique)
        #print(p)
        return 1. - np.sum(p**2)
    
    def _shannon_entropy(self,df):
        unique, counts = np.unique(df[self.y_name].values, return_counts=True)
        N = df[self.y_name].values.size
        p = counts/N
        if p <= 0.:
            H = 0
        else:
            H = -np.sum(p * np.log2(p))
        return H
        
    def _opt_fun(self,df,split_name):
        def fun(x):
            split_df = [df[df[split_name]<x],
                        df[df[split_name]>=x]]
            return self._loss(split_df[0]) + self._loss(split_df[1])
        return fun
        
    def _node_or_leaf(self,df):
        loss_parent = self._loss(df)
        if loss_parent < self.leaf_loss_threshold:
            return self._leaf(df)
        
        loss_best, split_df, split_threshold, split_name = self._loss_best(df)
        print(f"Computed split:\nloss: {loss_best:.2f} (parent: {loss_parent:.2f})\nattribute: {split_name}\nthreshold: {split_threshold}\ncount: {[len(df_.index) for df_ in split_df]}")
        if loss_best * self.splittyness < loss_parent:
            print(f"  => creating Node({split_name}, {split_threshold})\n")
            branches = []
            for i in range(2):
                branches.append(self._node_or_leaf(split_df[i]))
            item = Node(branches,split_name,split_threshold)
        else:
            item = self._leaf(df)
        return item
    
    def _leaf(self,df):
        unique, counts = np.unique(df[self.y_name].values,return_counts=True)
        print([(unique[i], counts[i]) for i in range(len(counts))])
        sort_ind = np.argsort(-counts)
        value = unique[sort_ind[0]]
        leaf = Leaf(value)
        
        # confusion matrix
        i_predict = self.classes.index(value)
        for i, c in enumerate(unique):
            i_c = self.classes.index(c)
            self.confusion_matrix[i_c,i_predict] += counts[i]
        
        print(f"  => creating Leaf({value}, N={len(df.index)})\n")
        return leaf
    
    def _loss_best(self,df):
        loss0 = 10
        for name in self.X_names:
            if np.issubdtype(df[name].values.dtype, np.number):
                #split_threshold_ = np.median(df[name].v
                res = opt.minimize_scalar(self._opt_fun(df,name),bounds=(df[name].min(),df[name].max()),method="bounded")
                split_threshold_ = res.x
                split_df_ = [df[df[name]<split_threshold_],
                        df[df[name]>=split_threshold_]]
                #loss = self._loss(split_df_[0]) + self._loss(split_df_[1])
                loss = res.fun
            else:
                unique = np.unique(df[name])
                split_threshold_ = [unique.ravel()[0]]
                split_df_ =[df[df[name].isin(split_threshold_)],
                            df[~df[name].isin(split_threshold_)]]
                loss = self._loss(split_df_[0]) + self._loss(split_df_[1])
            if loss < loss0:
                loss0 = loss
                split_threshold = split_threshold_
                split_df = split_df_
                split_name = name
                
        #print(loss0)
                
        return loss0, split_df, split_threshold, split_name
    
    def _loss(self,df):
        #return self._gini_impurity(df)
        return self._shannon_entropy(df)
    
    def metrics(self):
        P = self._precision(self.confusion_matrix)
        print(f"precision: {P}")
        R = self._recall(self.confusion_matrix)
        print(f"recall: {R}")
        F = np.mean(self._F1(P,R))
        print(f"F-score: {F}")
        return {"precision":P,
                "recall":R,
                "F-score":F}
    
    @staticmethod
    def _precision(m):
        return np.diag(m) / np.sum(m, axis=1)
        
    
    @staticmethod
    def _recall(m):
        return np.diag(m) / np.sum(m, axis=0)
    
    @staticmethod
    def _F1(P,R):
        #F = np.zeros_like(P)
        #for i in range(len(
        return 2 * P * R / (P + R)
            
        
        

In [ ]:
df=pd.read_csv("iris.csv")

In [ ]:
df.columns
X_names=["petal_length","petal_width"]
df[X_names]

In [ ]:
df.iloc[0]

In [ ]:
c = CART(df,"species",X_names)
c.create_tree(splittyness=1.)

In [ ]:
c.tree.predict(df.iloc[0]).value

In [ ]:
import matplotlib.pyplot as plt
colors = {"setosa":"red", "versicolor":"blue", "virginica":"green"}
plt.scatter(df["petal_length"],df["petal_width"],c=df["species"].map(colors))


In [ ]:

x, y = np.meshgrid(np.linspace(1,7,11),np.linspace(0,2.5,11))
col = []
for i in range(len(x.ravel())):
    d = df.iloc[120].copy()
    d["petal_length"] = x.ravel()[i]
    d["petal_width"] = y.ravel()[i]
    col.append(c.tree.predict(d).value)
for i in range(len(col)):
    if col[i] == "setosa":
        col[i] = 0
    if col[i] == "versicolor":
        col[i] = 1
    if col[i] == "virginica":
        col[i] = 2
z = np.array(col).reshape(x.shape)

In [ ]:
fig, ax = plt.subplots()
ax.pcolormesh(x,y,z)
ax.scatter(df["petal_length"],df["petal_width"],c=df["species"].map(colors))

In [ ]:
c.confusion_matrix
c.metrics()

In [ ]:
titanic = pd.read_csv("titanic.csv")
titanic

In [ ]:
titanic.columns

In [ ]:

c_titanic = CART(titanic,"Survived",["Pclass","Age","Fare","Siblings/Spouses Aboard","Sex","Parents/Children Aboard"])
c_titanic.create_tree(splittyness=0.95)

In [ ]:

c_titanic.metrics()

In [ ]:

penguins = pd.read_csv("penguins.txt").dropna()
penguins.columns

In [ ]:
penguins

In [ ]:
c_penguins = CART(penguins,"species",["island","bill_length_mm","bill_depth_mm","flipper_length_mm","body_mass_g","sex"])

In [ ]:
c_penguins.create_tree()

In [ ]:
c_penguins.metrics()